In [1]:
!pip install fastapi
!pip install kaleido
!pip install python-multipart
!pip install uvicorn
!pip install setuptools
!pip install simplet5

  Obtaining dependency information for fastapi from https://files.pythonhosted.org/packages/4d/d2/3ad038a2365fefbac19d9a046cab7ce45f4c7bfa81d877cbece9707de9ce/fastapi-0.103.2-py3-none-any.whl.metadata
  Obtaining dependency information for starlette<0.28.0,>=0.27.0 from https://files.pythonhosted.org/packages/58/f8/e2cca22387965584a409795913b774235752be4176d276714e15e1a58884/starlette-0.27.0-py3-none-any.whl.metadata
  Obtaining dependency information for typing-extensions>=4.5.0 from https://files.pythonhosted.org/packages/ec/6b/63cc3df74987c36fe26157ee12e09e8f9db4de771e0f3404263117e75b95/typing_extensions-4.7.1-py3-none-any.whl.metadata
  Using cached typing_extensions-4.7.1-py3-none-any.whl.metadata (3.1 kB)
   ---------------------------------------- 66.3/66.3 kB 890.4 kB/s eta 0:00:00
Using cached starlette-0.27.0-py3-none-any.whl (66 kB)
Using cached typing_extensions-4.7.1-py3-none-any.whl (33 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
confection 0.1.3 requires srsly<3.0.0,>=2.4.0, but you have srsly 1.0.7 which is incompatible.
confection 0.1.3 requires typing-extensions<4.5.0,>=3.7.4.1; python_version < "3.8", but you have typing-extensions 4.7.1 which is incompatible.
weasel 0.3.2 requires srsly<3.0.0,>=2.4.3, but you have srsly 1.0.7 which is incompatible.


  Using cached kaleido-0.2.1-py2.py3-none-win_amd64.whl (65.9 MB)
  Using cached python_multipart-0.0.6-py3-none-any.whl (45 kB)
  Using cached uvicorn-0.22.0-py3-none-any.whl (58 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 3.5/3.5 MB 1.9 MB/s eta 0:00:00
     ------------------------------------ 527.7/527.7 kB 736.1 kB/s eta 0:00:00
  Using cached future-0.18.3.tar.gz (840 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 519.2/519.2 kB 1.9 MB/s eta 0:00:00
     -------------------------------------- 952.4/952.4 kB 1.5 MB/s eta 0:00:00
     -------------------------------------- 880.6/880.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel 

DEPRECATION: pytorch-lightning 1.5.10 has a non-standard dependency specifier torch>=1.7.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:

import pandas as pd
df =  pd.read_csv("cnn_dataset/test.csv")

In [4]:
df["article_length"] =  df["article"].apply(lambda x : len(x.split()))
df["highlights_length"] =  df["highlights"].apply(lambda x : len(x.split()))
df =  df[df["article_length"] < 512 ]
df =  df[df["highlights_length"] < 128]
df.drop(columns=["article_length","highlights_length"],axis=1, inplace = True)
# simpleT5 expects dataframe to have 2 columns: "source_text" and "target_text"
df = df.rename(columns={"article":"target_text", "highlights":"source_text"})
df = df[['source_text', 'target_text']]
# T5 model expects a task related prefix: since it is a summarization task, we will add a prefix "summarize: "
df['source_text'] = "summarize: " + df['source_text']

In [5]:
from sklearn.model_selection   import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2)
train_df.shape, test_df.shape

((3392, 2), (848, 2))

In [8]:
from simplet5 import SimpleT5
model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")
model.train(train_df=train_df[:5000],
            eval_df=test_df[:100],
            source_max_token_len=2048,
            target_max_token_len=512,
            batch_size=4, max_epochs=3, use_gpu=False)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
f:\deeplearning_project\dl_project\lib\site-packages\pytorch_lightning\trainer\trainer.py:1585: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  "GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`."

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

f:\deeplearning_project\dl_project\lib\site-packages\pytorch_lightning\trainer\data_loading.py:133: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
Global seed set to 42
f:\deeplearning_project\dl_project\lib\site-packages\pytorch_lightning\trainer\data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Training: 0it [00:00, ?it/s]

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:76] data. DefaultCPUAllocator: not enough memory: you tried to allocate 805306368 bytes.

In [3]:
from simplet5 import SimpleT5

model = SimpleT5()

In [4]:
model.load_model("t5","model14epoch", use_gpu=True)

In [5]:
text =  """
Review: by mark prigg soldiers are set to get google glass like augmented reality system designed for the battlefield called arc4 it allows commanders to send maps and other information directly to the soldier field of the gadget attaches to military helmet and can even be integrated with weapons control system scroll down for video called arc4 the new system allows commanders to send maps and other information directly to the soldier field of the system can overlay key information such as the location of enemies satellite footage and mission objectives rather than looking down at 2d map or chest worn computer the soldier sees virtual icons such as navigation waypoints friendly blue forces and aircraft overlaid on their real world view the software will run on specially adapted display that clips onto helmets or on range of existing ar goggles it can track exactly where user is looking and adjust the display accordingly the firm behind it applied research associates says the system was developed as part of six year project with substantial investment from the us military darpa unit we worked closely with throughout the design analysis and testing phases of the program the firm said rather than looking down at 2d map or chest worn computer the soldier sees virtual icons such as navigation waypoints friendly blue forces and aircraft overlaid on their real world view you are able to perform your mission with high awareness of their surroundings with enhanced safety speed and in close coordination with team members ara claims this state of the art technology provides heads up situational awareness for the military allowing soldiers to perform their jobs with enhanced safety speed and mission effectiveness the firm said however it could also be used in logistics and even by the public how it works the system is connected to control units on the soldier body information is sent wirelessly to the soldier and displayed over one of their eyes as well as military uses the new system could be used for consumer products real world object tagging and sharing for socially networked users team coordination shared awareness for emergency management and response logistics and operations tracking assets personnel and tasks for and events immersive training and gaming virtual avatars and effects embedded with live operations the software will run on specially adapted display that clips onto helmets or on range of existing ar goggles it can track exactly where user is looking and adjust the display accordingly the gadget lead engineer dave roberts said it is flat out game changer that’s ideal for anyone who wants to visualize create and share heads up information arc4 will change the way you see and interact with the world around you the same way the smart phone changed the way you communicate the firm experimented with dozens of prototypes for the system before deciding on the final model left the arc4 software renders ar information on any wearable see through display including night vision devices making it easy to integrate with existing product the firm also says its system is more advanced that google glass arc does more than present conventional communications and directional information e g messages tweets maps pictures in heads up fashion arc4 is transforming your view of the world with digital tags that persist as if painted on physical objects imagine it as virtual
"""

In [6]:
len(text.split())

562

In [7]:
pred_text= model.predict(text)

Token indices sequence length is longer than the specified maximum sequence length for this model (636 > 512). Running this sequence through the model will result in indexing errors


In [8]:
type(pred_text)

list

In [11]:
len(pred_text[0].split())

123

In [9]:
pred_text[0]



'By mark prigg soldiers are set to get google glass like augmented reality system designed for the battlefield called arc4 it allows commanders to send maps and other information directly to the soldier field of the gadget attaches to their real world view. The firm said this is a flat out game changer that is ideal for anyone who wants to visualize create and share heads up information arc4 will change the way you see and interact with the world around you the same way the smart phone changed the way you communicate. It could also be used in logistics and even by the public how it works. The new system, which was developed as part of a six-year project with substantial investment from the us military darpa unit'

In [2]:
import pandas as pd

df = pd.read_csv("cnn_dataset/test.csv")

df["article_length"] = df["article"].apply(lambda x: len(x.split()))
df["highlights_length"] = df["highlights"].apply(lambda x: len(x.split()))
df = df[df["article_length"] <450 ]
text = df['article'][1]

In [14]:
pred_text[0]

'(CNN) -- A stray pooch in Washington State has been receiving care of her beloved cat after she was hit by a car, apparently whacked on the head with a hammer and then buried in a field just to survive. Theia is only one year old but her brush with death did not leave her unscathed. "She\'s a true miracle dog and she deserves a good life," Sara Mellado told WSU News Wednesday. "It\'s an amazing day for me. I\'m so grateful for all the support that we have received from the vet." She said: "It\'s hard to believe that it\'s going to be a great'

In [15]:
len(df)

3324

In [25]:
df['generated_highlights'] = df['article'].apply(lambda x: model.predict(x)[0])

KeyboardInterrupt: 

In [22]:
i = 0
text = df['article'][1]

In [20]:
df

,article,highlights,id,article_length,highlights_length,generated_highlights
1,(CNN)Never mind cats having nine lives. A stra...,"Theia, a bully breed mix, was apparently hit b...",230c522854991d053fe98a718b1defa077a8efef,411,44,
3,(CNN)Five Americans who were monitored for thr...,17 Americans were exposed to the Ebola virus w...,a38e72fed88684ec8d60dd5856282e999dc8c0ca,202,45,
4,(CNN)A Duke student has admitted to hanging a ...,Student is no longer on Duke University campus...,c27cf1b136cc270023de959e7ab24638021bc43f,364,43,
5,(CNN)He's a blue chip college basketball recru...,College-bound basketball star asks girl with D...,1b2cc634e2bfc6f2595260e7ed9b42f77ecbb0ce,440,24,
7,"(CNN)Andrew Getty, one of the heirs to billion...",Andrew Getty's death appears to be from natura...,0d3c8c276d079c4c225f034c69aa024cdab7869d,361,44,
...,...,...,...,...,...,...
11470,The German government had been warned of a hei...,German government 'knew of risk of flying over...,97bea23d3939621cb0504e61d2cbf1997938528f,423,40,
11473,Four children were rescued from their Philadel...,Single mother from Philadelphia is facing four...,7aefa3df2215604c3c0ecdecf51dd466bbb0fd03,403,64,
11479,Thousands of commuters were left stranded at W...,Thousands were left stranded at central London...,dc680726929ab6d5c2c493cde84cd0e5fdc13673,380,47,
11483,"Everyone knows the tortoise beat the hare, bu...",Amazing scene captured on film in South Africa...,e3b38ff7835f48c6933e78e9b7203a40164b5687,401,40,


In [26]:
import pandas as pd
from tqdm import tqdm

# Assuming df is your DataFrame and model.predict is your prediction function
# Replace model.predict with your actual prediction function

# Create a new column for generated highlights
df['generated_highlights'] = ""

# Use tqdm to display a progress bar
for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
    article_text = row['article']
    generated_highlight = model.predict(article_text)[0]
    df.at[index, 'generated_highlights'] = generated_highlight

Processing rows:  39%|███▉      | 1291/3324 [1:35:21<2:06:10,  3.72s/it]

In [ ]:
df.to_csv("generated_highlights.csv", index=False)

In [7]:

import openai

API_KEY = "YOUR_API_KEY"
API_ENDPOINT = "https://api.openai.com/v1/chat/completions"

In [8]:
import requests
import json

def generate_chat_completions(messages, model="gpt-4", temperature=0, max_tokens=None):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {API_KEY}",
    }

    responses = []  # Initialize an empty list to store responses

    for message in messages:
        data = {
            "model": model,
            "messages": message,
            "temperature": temperature,
        }

        if max_tokens is not None:
            data["max_tokens"] = max_tokens

        response = requests.post(API_ENDPOINT, headers=headers, data=json.dumps(data))

        if response.status_code == 200:
            response_text = response.json()["choices"][0]["message"]["content"]
            responses.append(response_text)
        else:
            raise Exception(f"Error {response.status_code}: {response.text}")

    return responses


In [10]:
def brief_function_summary(text):
    message_1 =  [[{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": f"I want you to briefly summarize the text under 120 words , here is the text   {text}, do not give any additional outputs other than the summary"}] ]
    return message_1

In [4]:
text = """by mark prigg soldiers are set to get google glass like augmented reality system designed for the battlefield called arc4 it allows commanders to send maps and other information directly to the soldier field of the gadget attaches to military helmet and can even be integrated with weapons control system scroll down for video called arc4 the new system allows commanders to send maps and other information directly to the soldier field of the system can overlay key information such as the location of enemies satellite footage and mission objectives rather than looking down at 2d map or chest worn computer the soldier sees virtual icons such as navigation waypoints friendly blue forces and aircraft overlaid on their real world view the software will run on specially adapted display that clips onto helmets or on range of existing ar goggles it can track exactly where user is looking and adjust the display accordingly the firm behind it applied research associates says the system was developed as part of six year project with substantial investment from the us military darpa unit we worked closely with throughout the design analysis and testing phases of the program the firm said rather than looking down at 2d map or chest worn computer the soldier sees virtual icons such as navigation waypoints friendly blue forces and aircraft overlaid on their real world view you are able to perform your mission with high awareness of their surroundings with enhanced safety speed and in close coordination with team members ara claims this state of the art technology provides heads up situational awareness for the military allowing soldiers to perform their jobs with enhanced safety speed and mission effectiveness the firm said however it could also be used in logistics and even by the public how it works the system is connected to control units on the soldier body information is sent wirelessly to the soldier and displayed over one of their eyes as well as military uses the new system could be used for consumer products real world object tagging and sharing for socially networked users team coordination shared awareness for emergency management and response logistics and operations tracking assets personnel and tasks for and events immersive training and gaming virtual avatars and effects embedded with live operations the software will run on specially adapted display that clips onto helmets or on range of existing ar goggles it can track exactly where user is looking and adjust the display accordingly the gadget lead engineer dave roberts said it is flat out game changer that’s ideal for anyone who wants to visualize create and share heads up information arc4 will change the way you see and interact with the world around you the same way the smart phone changed the way you communicate the firm experimented with dozens of prototypes for the system before deciding on the final model left the arc4 software renders ar information on any wearable see through display including night vision devices making it easy to integrate with existing product the firm also says its system is more advanced that google glass arc does more than present conventional communications and directional information e g messages tweets maps pictures in heads up fashion arc4 is transforming your view of the world with digital tags that persist as if painted on physical objects imagine it as virtual"""

In [11]:
gen_sum = generate_chat_completions(brief_function_summary(text), model="gpt-4", temperature=0)

Exception: Error 429: {
    "error": {
        "message": "You exceeded your current quota, please check your plan and billing details.",
        "type": "insufficient_quota",
        "param": null,
        "code": "insufficient_quota"
    }
}


In [13]:
gen_sum[0]

'A stray dog in Washington State, named Theia, survived being hit by a car, an attempted mercy killing with a hammer, and being buried in a field. She was found four days later by a farm worker who took her to a vet. Theia was taken in by Sara Mellado, who described her as "gentle and loving". Theia, who is only a year old, suffered a dislocated jaw, leg injuries, and a caved-in sinus cavity. She still requires surgery. Mellado has set up a fundraising page to help with the cost of Theia\'s care, which has already surpassed its $10,000 target.'

In [16]:
#now get summaries for the first 200 articles\
    
import tqdm
df  =  df.head(200)

df['generated_highlights_gpt4'] = ""

# Use tqdm to display a progress bar
for index, row in tqdm.tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
    article_text = row['article']
    generated_highlight = generate_chat_completions(brief_function_summary(article_text), model="gpt-4", temperature=0)[0]
    df.at[index, 'generated_highlights_gpt4'] = generated_highlight

Processing rows: 100%|██████████| 200/200 [20:45<00:00,  6.23s/it]


In [17]:
df.to_csv("generated_highlights_gpt4.csv", index=False)

In [2]:
import pandas as pd
df = pd.read_csv("generated_highlights_gpt4.csv")

In [3]:
df

,article,highlights,id,article_length,highlights_length,generated_highlights_gpt4
0,(CNN)Never mind cats having nine lives. A stra...,"Theia, a bully breed mix, was apparently hit b...",230c522854991d053fe98a718b1defa077a8efef,411,44,"A stray dog in Washington State, named Theia, ..."
1,(CNN)Five Americans who were monitored for thr...,17 Americans were exposed to the Ebola virus w...,a38e72fed88684ec8d60dd5856282e999dc8c0ca,202,45,"Five Americans, who were exposed to Ebola in S..."
2,(CNN)A Duke student has admitted to hanging a ...,Student is no longer on Duke University campus...,c27cf1b136cc270023de959e7ab24638021bc43f,364,43,A Duke University student admitted to hanging ...
3,(CNN)He's a blue chip college basketball recru...,College-bound basketball star asks girl with D...,1b2cc634e2bfc6f2595260e7ed9b42f77ecbb0ce,440,24,"Trey Moses, a top college basketball recruit, ..."
4,"(CNN)Andrew Getty, one of the heirs to billion...",Andrew Getty's death appears to be from natura...,0d3c8c276d079c4c225f034c69aa024cdab7869d,361,44,"Andrew Getty, an heir to the Getty oil fortune..."
...,...,...,...,...,...,...
195,(CNN)A family trip to a Nebraska zoo turned te...,Gorilla leaps toward exhibit window and hits i...,dbce61d253b9e770529817b484aeb8b0cca76a73,232,18,A family's visit to Omaha's Henry Doorly Zoo i...
196,"(CNN)Looks like another really, really, ridicu...",Ben Stiller announces that Penelope Cruz will ...,844f3422fe9669127bd178ab7293aa59bde3da06,150,21,Actress Penelope Cruz is set to join the cast ...
197,(CNN)A SkyWest Airlines flight made an emergen...,FAA backtracks on saying crew reported a press...,1a9fb7c99082836c6a41091f38c1914c51c34e4f,180,24,A SkyWest Airlines flight made an emergency la...
198,(CNN)Cigarettes have been put out across the b...,New Orleans bars are smoke-free as of Wednesda...,54885ae2f80774083a4489cf39c0526d7a2c285f,186,23,"New Orleans, a city known for its indulgence a..."


In [4]:
#get rogue scores for the generated summaries

from rouge import Rouge

rouge = Rouge()


def get_rouge_scores(generated_highlights, highlights):
    scores = rouge.get_scores(generated_highlights, highlights)
    return scores

df['rouge_scores'] = df.apply(lambda x: get_rouge_scores(x['generated_highlights_gpt4'], x['highlights']), axis=1)

In [21]:
df['rouge_scores']

0      [{'rouge-1': {'r': 0.47058823529411764, 'p': 0...
1      [{'rouge-1': {'r': 0.4857142857142857, 'p': 0....
2      [{'rouge-1': {'r': 0.6111111111111112, 'p': 0....
3      [{'rouge-1': {'r': 0.5238095238095238, 'p': 0....
4      [{'rouge-1': {'r': 0.5405405405405406, 'p': 0....
                             ...                        
195    [{'rouge-1': {'r': 0.3333333333333333, 'p': 0....
196    [{'rouge-1': {'r': 0.7777777777777778, 'p': 0....
197    [{'rouge-1': {'r': 0.7142857142857143, 'p': 0....
198    [{'rouge-1': {'r': 0.5714285714285714, 'p': 0....
199    [{'rouge-1': {'r': 0.5769230769230769, 'p': 0....
Name: rouge_scores, Length: 200, dtype: object

In [5]:
#get the average rouge scores for the generated summaries rouge-1, rouge-2, rouge-l
df['rouge_scores'].apply(lambda x: x[0]['rouge-1']['f']).mean()



0.29571237638789805

In [6]:
df['rouge_scores'].apply(lambda x: x[0]['rouge-2']['f']).mean()

0.1044669459816257

In [8]:
df['rouge_scores'].apply(lambda x: x[0]['rouge-l']['f']).mean()

0.2752779224370842